In [2]:
import sys

# 清除任何可能导致冲突的路径
for i in range(len(sys.path)-1, -1, -1):
    if "StableDiffusionReconstruction" in sys.path[i] or "diffusion_sd1" in sys.path[i]:
        sys.path.pop(i)

# 添加正确的路径
sys.path.insert(0, '/data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/decoding_sd2/stablediffusion')
print("当前Python路径:", sys.path)

当前Python路径: ['/data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/decoding_sd2/stablediffusion', '/data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/decoding_sd2/stablediffusion', '/data/zlhu', '/data/zlhu/miniconda3/envs/brain/lib/python38.zip', '/data/zlhu/miniconda3/envs/brain/lib/python3.8', '/data/zlhu/miniconda3/envs/brain/lib/python3.8/lib-dynload', '', '/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages', '/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/bdpy-0.18-py3.8.egg']


In [3]:
import argparse, os
from tqdm import tqdm, trange
from torch import autocast
from contextlib import nullcontext
import torch
import numpy as np
import pandas as pd
from PIL import Image
from omegaconf import OmegaConf
from einops import repeat, rearrange
from pytorch_lightning import seed_everything
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.data.util import AddMiDaS


/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


文件已存在


In [4]:
def initialize_model(config, ckpt,device):
    config = OmegaConf.load(config)
    model = instantiate_from_config(config.model)
    model.load_state_dict(torch.load(ckpt)["state_dict"], strict=False)
    model = model.to(device)
    sampler = DDIMSampler(model)
    return sampler
    
def initialize_model(config, ckpt, device):
    config = OmegaConf.load(config)
    print(f"模型目标: {config.model.target}")
    
    # 直接导入正确路径的模块
    from stablediffusion.ldm.util import instantiate_from_config
    model = instantiate_from_config(config.model)
    model.load_state_dict(torch.load(ckpt)["state_dict"], strict=False)
    model = model.to(device)
    from stablediffusion.ldm.models.diffusion.ddim import DDIMSampler
    sampler = DDIMSampler(model)
    return sampler
def pad_image(input_image):
    pad_w, pad_h = np.max(((2, 2), np.ceil(
        np.array(input_image.size) / 64).astype(int)), axis=0) * 64 - input_image.size
    im_padded = Image.fromarray(
        np.pad(np.array(input_image), ((0, pad_h), (0, pad_w), (0, 0)), mode='edge'))
    return im_padded


def make_batch_sd(
        image,
        txt,
        device,
        num_samples=1,
        model_type="dpt_hybrid"
):
    image = np.array(image.convert("RGB"))
    image = torch.from_numpy(image).to(dtype=torch.float32) / 127.5 - 1.0
    # sample['jpg'] is tensor hwc in [-1, 1] at this point
    midas_trafo = AddMiDaS(model_type=model_type)
    batch = {
        "jpg": image,
        "txt": txt,
    }
    batch = midas_trafo(batch)
    batch["jpg"] = rearrange(batch["jpg"], 'h w c -> 1 c h w')
    batch["jpg"] = repeat(batch["jpg"].to(device=device),
                          "1 ... -> n ...", n=num_samples)
    batch["midas_in"] = repeat(torch.from_numpy(batch["midas_in"][None, ...]).to(
        device=device), "1 ... -> n ...", n=num_samples)
    return batch


In [5]:
seed_everything(42)
imgidxs = [0, 10]
gpu = 0
torch.cuda.set_device(gpu)
subject='subj01'
config = './stablediffusion/configs/stable-diffusion/v2-midas-inference.yaml'
ckpt = './stablediffusion/models/512-depth-ema.ckpt'
steps = 50
scale = 5.0
eta = 0.0
strength = 0.8
num_samples= 1
callback=None
n_iter = 5

Global seed set to 42


In [6]:
precision = 'autocast'
precision_scope = autocast if precision == "autocast" else nullcontext

In [7]:
import os
# 获取当前文件所在目录
current_dir = os.path.dirname(os.path.abspath(''))
print("当前工作目录:", current_dir)
# 设置工作目录
os.chdir('/data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/decoding_sd2')
# 验证工作目录
print("当前工作目录:", os.getcwd())

当前工作目录: /data
当前工作目录: /data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/decoding_sd2


In [8]:
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
sampler = initialize_model(config,ckpt,device)
model = sampler.model

模型目标: ldm.models.diffusion.ddpm.LatentDepth2ImageDiffusion
文件已存在
No module 'xformers'. Proceeding without it.
LatentDepth2ImageDiffusion: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any fe

In [9]:
'''采样策略'''
sampler.make_schedule(ddim_num_steps=steps, ddim_eta=eta, verbose=True)

assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'

t_enc = min(int(strength * steps), steps-1)
print(f"target t_enc is {t_enc} steps")


Selected timesteps for ddim sampler: [  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
Selected alphas for ddim sampler: a_t: tensor([0.9983, 0.9804, 0.9609, 0.9398, 0.9171, 0.8930, 0.8674, 0.8404, 0.8121,
        0.7827, 0.7521, 0.7207, 0.6885, 0.6557, 0.6224, 0.5888, 0.5551, 0.5215,
        0.4882, 0.4552, 0.4229, 0.3913, 0.3605, 0.3308, 0.3023, 0.2750, 0.2490,
        0.2245, 0.2014, 0.1799, 0.1598, 0.1413, 0.1243, 0.1087, 0.0946, 0.0819,
        0.0705, 0.0604, 0.0514, 0.0435, 0.0365, 0.0305, 0.0254, 0.0210, 0.0172,
        0.0140, 0.0113, 0.0091, 0.0073, 0.0058]); a_(t-1): [0.99914998 0.99829602 0.98038077 0.96087277 0.93978298 0.91713792
 0.89298052 0.86737001 0.84038192 0.81210774 0.78265446 0.75214338
 0.72070938 0.68849909 0.65566933 0.62238538 0.58881873 0.55514455
 0.52153981 0.4881804  0.45523876 0.42288151 0.39126703 0.36

In [10]:
# Load Prediction (C, InitLatent, Depth(cc))
captdir = f'../../Brain-Caption/decoded_captions/{subject}'
dptdir = f'../../data/decoded/{subject}/dpt_fromemb/'
# gandir = f'../../decoded/gan_recon_img/all_layers/{subject}/streams/'

# C
captions = pd.read_csv(f'{captdir}/captions_brain.csv', sep='\t',header=None)


In [11]:
import h5py
import scipy.io
from nsd_access.nsda import NSDAccess
# Load NSD information

nsd_expdesign = scipy.io.loadmat('../../data/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

nsda = NSDAccess('../../data/nsd/')
sf = h5py.File(nsda.stimuli_file, 'r')
sdataset = sf.get('imgBrick')

stims_ave = np.load(f'../../data/stim/{subject}/{subject}_stims_ave.npy')

# Note that mos of them are 1-base index!
# This is why I subtract 1
sharedix = nsd_expdesign['sharedix'] -1 
tr_idx = np.zeros_like(stims_ave)
for idx, s in enumerate(stims_ave):
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1

In [11]:
### decoding NO.1 第一版

# Save Directories
outdir = f'../../data/decoded/{subject}/image-depth-v1/'
os.makedirs(outdir, exist_ok=True)
sample_path = os.path.join(outdir, f"samples")
os.makedirs(sample_path, exist_ok=True)

# padding-batch
for imgidx in range(10):
    
    print(f"now is img{imgidx}.../n")
    # Load z (Image)
    imgidx_te = np.where(tr_idx==0)[0][imgidx] # Extract test image index
    idx73k= stims_ave[imgidx_te]
    Image.fromarray(np.squeeze(sdataset[idx73k,:,:,:]).astype(np.uint8)).save(
        os.path.join(sample_path, f"{imgidx:05}_org.png"))
    
    # get caption and depth
    prompt = [captions.iloc[imgidx][0]]
    cc = torch.Tensor(np.load(f'{dptdir}/{imgidx:06}.npy')).to('cuda')
    
    # # Generate image from Text + GAN + Depth
    # shenpath = f'{gandir}/recon_image_normalized-VGG19-fc8-{subject}-streams-{imgidx:06}.tiff'
    # init_image = Image.open(shenpath).resize((512,512))
    
    # Generate image from Text + Depth
    
    roi_latent = 'early'
    scores_latent = np.load(f'../data/decoded/{subject}/{subject}_{roi_latent}_brain_embs_init_latent.npy')
    imgarr = torch.Tensor(scores_latent[imgidx,:].reshape(4,40,40)).unsqueeze(0).to('cuda')
    
    # Generate image from Z
    precision_scope = autocast if precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                x_samples = model.decode_first_stage(imgarr)
                x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                for x_sample in x_samples:
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
    init_image = Image.fromarray(x_sample.astype(np.uint8)).resize((512,512))
    image = pad_image(init_image)
    
    base_count = 0
    with torch.no_grad():
        for n in trange(n_iter, desc="Sampling"):
            torch.autocast("cuda")
            batch = make_batch_sd(image, txt=prompt, device=device, num_samples=num_samples)
            z = model.get_first_stage_encoding(model.encode_first_stage(
                batch[model.first_stage_key]))  # move to latent space
            c = model.cond_stage_model.encode(batch["txt"]).mean(axis=0).unsqueeze(0)
            c_cat = list()
            c_cat.append(cc)
            c_cat = torch.cat(c_cat, dim=1)
    
            # cond
            cond = {"c_concat": [c_cat], "c_crossattn": [c]}
    
            # uncond cond
            uc_cross = model.get_unconditional_conditioning(num_samples, "")
            uc_full = {"c_concat": [c_cat], "c_crossattn": [uc_cross]}
    
            # encode (scaled latent)
            z_enc = sampler.stochastic_encode(
                z, torch.tensor([t_enc] * num_samples).to(model.device))
    
            # decode it
            samples = sampler.decode(z_enc, cond, t_enc, unconditional_guidance_scale=scale,
                                    unconditional_conditioning=uc_full, callback=callback)
            x_samples_ddim = model.decode_first_stage(samples)
            result = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
            result = result.cpu().numpy().transpose(0, 2, 3, 1) * 255
            Image.fromarray(result[0,:,:,:].astype(np.uint8)).save(
                os.path.join(sample_path, f"{imgidx:05}_{base_count:03}.png"))   
            base_count += 1

now is img0.../n


NameError: name 'tr_idx' is not defined

In [13]:
### decoding NO.2 第二版
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def load_img_from_arr(img_arr):
    image = Image.fromarray(img_arr).convert("RGB")
    w, h = 512, 512
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

# Save Directories
outdir = f'../../data/decoded/{subject}/image-depth-v2/'
os.makedirs(outdir, exist_ok=True)
sample_path = os.path.join(outdir, f"samples")
os.makedirs(sample_path, exist_ok=True)

for imgidx in range(10):

    c = model.get_learned_conditioning(captions.iloc[imgidx][0]).to('cuda')
    cc = torch.Tensor(np.load(f'{dptdir}/{imgidx:06}.npy')).to('cuda')
    
    # # Generate image from Text + GAN + Depth
    # shenpath = f'{gandir}/recon_image_normalized-VGG19-fc8-{subject}-streams-{imgidx:06}.tiff'
    # init_image = Image.open(shenpath).resize((512,512))
    
    # Generate image from Text + Depth
    
    roi_latent = 'early'
    scores_latent = np.load(f'../data/decoded/{subject}/{subject}_{roi_latent}_brain_embs_init_latent.npy')
    imgarr = torch.Tensor(scores_latent[imgidx,:].reshape(4,40,40)).unsqueeze(0).to('cuda')
    
    # Generate image from Z
    precision_scope = autocast if precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                x_samples = model.decode_first_stage(imgarr)
                x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                for x_sample in x_samples:
                    x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
    
    im = Image.fromarray(x_sample.astype(np.uint8)).resize((512,512))
    im = np.array(im)
    init_image = load_img_from_arr(im).to('cuda')
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
    
    base_count = 0
    with torch.no_grad():
        for n in trange(n_iter, desc="Sampling"):
            with torch.autocast("cuda"):
                c_cat = list()
                c_cat.append(cc)
                c_cat = torch.cat(c_cat, dim=1)
        
                # cond
                cond = {"c_concat": [c_cat], "c_crossattn": [c]}
        
                # uncond cond
                uc_cross = model.get_unconditional_conditioning(num_samples, "")
                uc_full = {"c_concat": [c_cat], "c_crossattn": [uc_cross]}
        
                # encode (scaled latent)
                z_enc = sampler.stochastic_encode(
                    init_latent, torch.tensor([t_enc] * num_samples).to(model.device))
        
                # decode it
                samples = sampler.decode(z_enc, cond, t_enc, unconditional_guidance_scale=scale,
                                        unconditional_conditioning=uc_full, callback=callback)
                x_samples_ddim = model.decode_first_stage(samples)
                result = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                result = result.cpu().numpy().transpose(0, 2, 3, 1) * 255
                Image.fromarray(result[0,:,:,:].astype(np.uint8)).save(
                    os.path.join(sample_path, f"{imgidx:05}_{base_count:03}.png"))   
                base_count += 1

Sampling:   0%|                                                         | 0/5 [00:00<?, ?it/s]

Running DDIM Sampling with 40 timesteps


Sampling:  20%|█████████▊                                       | 1/5 [00:13<00:52, 13.11s/it]

Running DDIM Sampling with 40 timesteps


Sampling:  40%|███████████████████▌                             | 2/5 [00:26<00:39, 13.00s/it]

...

Running DDIM Sampling with 40 timesteps


Sampling:  80%|███████████████████████████████████████▏         | 4/5 [00:52<00:13, 13.13s/it]

Running DDIM Sampling with 40 timesteps


Sampling: 100%|█████████████████████████████████████████████████| 5/5 [01:05<00:00, 13.13s/it]


: 